In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from string import ascii_lowercase
from itertools import combinations

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [165]:
train = pd.read_csv('./data/train.csv').drop('index',axis=1)
test = pd.read_csv('./data/test.csv').drop('index',axis=1)

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)

In [167]:
train.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


In [168]:
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])

In [169]:
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

In [170]:
train_fill_na = train.fillna(train.mean())

In [171]:
y_train = train_fill_na['nerdiness']
x_train = train_fill_na.drop('nerdiness',axis=1)

In [172]:
k_fold = KFold(n_splits = 3, shuffle = True, random_state = 0)

In [173]:
from catboost import CatBoostClassifier

params = {'bagging_temperature': 0.375906, 'depth': 9.0, 'l2_leaf_reg': 68.8, 'learning_rate': 0.011, 'subsample': 0.76046}

clf1 = CatBoostClassifier(**params, iterations=5000, eval_metric='AUC',allow_writing_files=False,od_type='Iter',random_state=777)
clf2 = LGBMClassifier()
clf3 = GradientBoostingClassifier()
soft_vote  = VotingClassifier([('r1',clf1), ('r2', clf2), ('r3',clf3)], voting='soft')
soft_vote.fit(x_train, y_train)

0:	total: 27.1ms	remaining: 2m 15s
1:	total: 53.9ms	remaining: 2m 14s
2:	total: 78.5ms	remaining: 2m 10s
3:	total: 104ms	remaining: 2m 9s
4:	total: 128ms	remaining: 2m 7s
5:	total: 151ms	remaining: 2m 5s
6:	total: 174ms	remaining: 2m 4s
7:	total: 199ms	remaining: 2m 4s
8:	total: 221ms	remaining: 2m 2s
9:	total: 244ms	remaining: 2m 1s
10:	total: 267ms	remaining: 2m
11:	total: 289ms	remaining: 2m
12:	total: 313ms	remaining: 2m
13:	total: 334ms	remaining: 1m 58s
14:	total: 355ms	remaining: 1m 57s
15:	total: 366ms	remaining: 1m 53s
16:	total: 386ms	remaining: 1m 53s
17:	total: 407ms	remaining: 1m 52s
18:	total: 426ms	remaining: 1m 51s
19:	total: 446ms	remaining: 1m 50s
20:	total: 464ms	remaining: 1m 50s
21:	total: 484ms	remaining: 1m 49s
22:	total: 503ms	remaining: 1m 48s
23:	total: 523ms	remaining: 1m 48s
24:	total: 544ms	remaining: 1m 48s
25:	total: 563ms	remaining: 1m 47s
26:	total: 582ms	remaining: 1m 47s
27:	total: 600ms	remaining: 1m 46s
28:	total: 618ms	remaining: 1m 45s
29:	total: 

VotingClassifier(estimators=[('r1',
                              <catboost.core.CatBoostClassifier object at 0x7f4b8c26c4f0>),
                             ('r2', LGBMClassifier()),
                             ('r3', GradientBoostingClassifier())],
                 voting='soft')

In [209]:
value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test_fill_na = test.fillna(test.mean())

In [176]:
model = soft_vote
pred_y = model.predict_proba(test_fill_na)
pred_y = pred_y[:,1]

index = test_index

submission = pd.DataFrame({
    "index" : index,
    "voted" : pred_y
})
submission.to_csv('./data/model1.csv', index=False)

# model2

In [177]:
y_train = train['nerdiness']
x_train = train.drop('nerdiness',axis=1)

In [178]:
import pandas as pd
import numpy as np

from string import ascii_lowercase
from itertools import combinations

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

In [179]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
import eli5
from eli5.sklearn import PermutationImportance

import matplotlib.pyplot as plt

import warnings
import gc
warnings.filterwarnings("ignore")

In [180]:
def lgbm_rfe_777(x_data, y_data, ratio=0.95, min_feats=20):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=777)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [181]:
lgbm_archive_4040 = lgbm_rfe_777(x_train, y_train)

73 0.853803694242032
69 0.8519897423044288
65 0.8539040062728074
61 0.8461952135352834
57 0.8506862607583934
54 0.8498688183070319
51 0.8488222965189138
48 0.8499820645538099
45 0.8481896696428776
42 0.8482635383876034
39 0.8476725884297978
37 0.8439446599508384
35 0.8464633829470697
33 0.8444056928945741
31 0.8385562655641733
29 0.8390356938373347
27 0.8383507552432437
25 0.8308940360042585
23 0.8249491829024611
21 0.8243033343834001


In [182]:
model = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_1 = x_train[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]]

model.fit(x_train_1, y_train)

pred_y1 = model.predict_proba(test[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]])
pred_y1 = pred_y1[:,1]

In [183]:
def lgbm_rfe_1234(x_data, y_data, ratio=0.95, min_feats=20):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [184]:
lgbm_archive_1234 = lgbm_rfe_1234(x_train, y_train)

73 0.8329435600645396
69 0.8340870226761852
65 0.8363914733314639
61 0.8358033658023085
57 0.8426627049613636
54 0.8403651495580143
51 0.8394759493612698
48 0.8383586312465237
45 0.8382460087983414
42 0.833022855461729
39 0.8335833819831663
37 0.8302848658413817
35 0.8284797463466657
33 0.8275937064737222
31 0.8270771371833355
29 0.8246324830721565
27 0.814057177727417
25 0.822569366234411
23 0.8136954643032808
21 0.8124172569768457


In [185]:
model2 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_2 = x_train[lgbm_archive_1234.iloc[lgbm_archive_1234[lgbm_archive_1234['score']==lgbm_archive_1234['score'].max()].index[0],2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[lgbm_archive_1234.iloc[lgbm_archive_1234[lgbm_archive_1234['score']==lgbm_archive_1234['score'].max()].index[0],2]])
pred_y2 = pred_y2[:,1]

In [186]:
def lgbm_rfe_99087(x_data, y_data, ratio=0.95, min_feats=20):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [187]:
lgbm_archive_99087 = lgbm_rfe_99087(x_train, y_train)

73 0.8404031076571365
69 0.8409772261538144
65 0.8423954167117068
61 0.8414492855286491
57 0.8335685963794599
54 0.8316041739018438
51 0.8344759163471689
48 0.8367364540233718
45 0.8311841503046249
42 0.8311082528168318
39 0.8384266105791898
37 0.8319089138149527
35 0.8300847866935605
33 0.8338767861783776
31 0.8313385326945678
29 0.8281997115895463
27 0.8207726824242119
25 0.820481742054338
23 0.8138637111111622
21 0.8122871132965499


In [188]:
model3 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_3 = x_train[lgbm_archive_99087.iloc[lgbm_archive_99087[lgbm_archive_99087['score']==lgbm_archive_99087['score'].max()].index[0],2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[lgbm_archive_99087.iloc[lgbm_archive_99087[lgbm_archive_99087['score']==lgbm_archive_99087['score'].max()].index[0],2]])
pred_y3 = pred_y3[:,1]

In [189]:
def lgbm_rfe_42(x_data, y_data, ratio=0.95, min_feats=20):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [190]:
lgbm_archive_42 = lgbm_rfe_42(x_train, y_train)

73 0.8470160165046442
69 0.8487083872512674
65 0.841944660627858
61 0.8463212234804353
57 0.8449808543363835
54 0.8414942252165543
51 0.8456684519067377
48 0.8473729110158368
45 0.8416597206552121
42 0.8408077789188169
39 0.842857331994787
37 0.8452793217622759
35 0.8365941211415098
33 0.8429750496198497
31 0.8413295932323591
29 0.8393917136002136
27 0.835732393587872
25 0.8350991936486594
23 0.831819505781691
21 0.8294562309176564


In [191]:
model4 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_4 = x_train[lgbm_archive_42.iloc[lgbm_archive_42[lgbm_archive_42['score']==lgbm_archive_42['score'].max()].index[0],2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[lgbm_archive_42.iloc[lgbm_archive_42[lgbm_archive_42['score']==lgbm_archive_42['score'].max()].index[0],2]])
pred_y4 = pred_y4[:,1]

In [192]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)

submission = pd.DataFrame({
    "index" : index,
    "voted" : pred_all
})
submission.to_csv('./data/model2.csv', index=False)

# model3

In [271]:
train_fill_na

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness,Ex,Ag,Con,Es,Op
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,1,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1,4.5,4.0,4.0,3.0,3.0
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,1,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1,3.0,3.5,2.0,3.5,2.5
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,2,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1,3.0,2.5,3.5,3.0,3.5
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,1,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1,3.0,3.5,3.5,3.5,4.0
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,2,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0,3.5,3.0,4.0,3.5,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,2.0,5.0,4.0,2.0,4.0,1,12,103,161,2.0,2.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,2.0,2,2.0,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0,3.5,2.5,3.5,3.0,3.0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,1.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,1.0,5.0,3.0,4.0,4.0,5.0,4.0,5.0,1,31,106,179,3.0,2.0,4.0,5.0,4.0,3.0,4.0,1.0,2.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,4.0,1,2.0,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1,3.0,3.0,2.5,3.5,3.0
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0,1,17,103,168,1.0,3.0,2.0,5.0,1.0,5.0,3.0,3.0,1.0,1.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1,3.0,3.0,2.5,3.0,1.0
14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,3.0,5.0,1,14,68,109,1.0,1.0,3.0,5.0,4.0,5.0,5.0,4.0,2.0,1.0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,3.0,2,2.0,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0,3.0,3.0,3.5,3.5,2.5


In [267]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_x = train_fill_na.drop('nerdiness',axis=1)
test_x = test_fill_na.copy()

train_y = y_train.copy()

# train_y = 2 - train_y.to_numpy()
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()

train_y_t = torch.tensor(train_y, dtype=torch.float32)
train_x_t = torch.tensor(train_x, dtype=torch.float32)
test_x_t = torch.tensor(test_x, dtype=torch.float32)

# train_x_t[:, :26] = (train_x_t[:, :26] - 3.) / 2.
# test_x_t[:, :26] = (test_x_t[:, :26] - 3.) / 2

# train_x_t[:, 30:40] = (train_x_t[:, 30:40] - 3.) / 2.
# test_x_t[:, 30:40] = (test_x_t[:, 30:40] - 3.) / 2

test_len = len(test_x_t)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 48
BATCH_SIZE = 72
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(train_x, train_y)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x_t[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x_t[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x_t, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
            nn.Dropout(0.05),
            nn.Linear(73, 180, bias=False),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(180, 32, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1)
        ).to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=4e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                loss.backward()
                optimizer.step()
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'.format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))

df = pd.read_csv('./data/sample_submission.csv')
df.iloc[:, 1:] = prediction

07/07: 100%|██████████| 48/48 [00:37<00:00,  1.28it/s]


R1 -> 1.0000


07/07: 100%|██████████| 48/48 [00:37<00:00,  1.28it/s]


R2 -> 0.9819


07/07: 100%|██████████| 48/48 [00:37<00:00,  1.28it/s]


R3 -> 1.0000


07/07: 100%|██████████| 48/48 [00:37<00:00,  1.28it/s]


R4 -> 1.0000


07/07: 100%|██████████| 48/48 [00:36<00:00,  1.30it/s]

R5 -> 1.0000


In [270]:
for i in df['nerdiness']:
    print(i)

0.0364142581820488
0.03969961404800415
0.03824710100889206
0.03687090054154396
0.03645765408873558
0.03907270357012749
0.041706573218107224
0.037880875170230865
0.043465275317430496
0.031224694103002548
0.04234153404831886
0.038810934871435165
0.040490567684173584
0.03166503086686134
0.042105428874492645
0.04433000832796097
0.03949275240302086
0.04394545033574104
0.0398566871881485
0.037875354290008545
0.03977541625499725
0.03841777518391609
0.03971296548843384
0.04000599682331085
0.04171951860189438
0.03969527781009674
0.03942839056253433
0.03859872743487358
0.042266231030225754
0.03932025656104088
0.03961551561951637
0.04230133816599846
0.04012482613325119
0.037654392421245575
0.041142743080854416
0.03716889023780823
0.042154666036367416
0.042384952306747437
0.03878260776400566
0.040441323071718216
0.04156232625246048
0.03811093047261238
0.035903386771678925
0.04123394936323166
0.03822729364037514
0.040870558470487595
0.039968620985746384
0.04173523932695389
0.038116924464702606
0.03

In [196]:
df.to_csv('./data/model3.csv', index=False)

# final Ensemble

In [197]:
model1 = pd.read_csv('./data/model1.csv', index_col = 'index')
model2 = pd.read_csv('./data/model2.csv', index_col='index')

pred_y = (model1)*(0.7) + (model2)*(0.3)

test = pd.read_csv('./data/test.csv')
index = test['index']

submission = pd.DataFrame({
    'index': index,
    'voted': pred_y['voted']
    })

submission.to_csv('./data/combined_model1_model2.csv', index=False)

In [ ]:
combined_12 = pd.read_csv('./data/combined_model1_model2.csv', index_col = 'index')
model3 = pd.read_csv('./data/model3.csv', index_col='index')
model3['voted'] = model3['voted']-1

In [ ]:
pred_y = (model3)*(0.8) + (combined_12)*(0.2)

test = pd.read_csv('./data/test_x.csv')
index = test['index']

submission = pd.DataFrame({
    'index': index,
    'voted': pred_y['voted']
    })

submission.to_csv('./data/submission_final_fix_lgbm.csv', index=False)